# Exemplo com Vassoura e dataset PD Behavior

Este notebook demonstra como carregar o dataset `pd_behavior_example.parquet`
 e aplicar a classe `Vassoura`.

In [1]:
import pandas as pd
from vassoura.core import Vassoura
import vassoura as vs
from audittrail import AuditTrail

pd.set_option('display.max_columns', None)

FILE_PATH_1 = '../../../datasets/lending_club/accepted_2007_to_2018Q4.csv'
FILE_PATH_2 = '../../../datasets/lending_club/rejected_2007_to_2018Q4.csv'

In [2]:
# Carregar dataset de exemplo
def read_and_clean_csv_mixed_types(path, nrows=None, verbose=True):
    """
    Lê um arquivo CSV, detecta colunas com tipos mistos e aplica conversão automática.
    
    Parâmetros:
    - path: caminho para o arquivo CSV
    - nrows: número de linhas a serem lidas (None = todas)
    - verbose: se True, imprime colunas com tipos mistos

    Retorna:
    - DataFrame limpo
    - Dicionário com as colunas que tinham tipos mistos
    """
    # Leitura inicial
    df = pd.read_csv(path, low_memory=False, nrows=nrows)

    # Identificar colunas com tipos mistos
    mixed_type_columns = {}
    for col in df.columns:
        types_in_col = df[col].dropna().apply(type).value_counts()
        if len(types_in_col) > 1:
            mixed_type_columns[col] = types_in_col
            if verbose:
                print(f"\n[!] Coluna '{col}' tem múltiplos tipos:")
                print(types_in_col)

    # Tentativa de padronização
    for col in mixed_type_columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para float.")
        except Exception:
            df[col] = df[col].astype(str)
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para string.")

    return df, mixed_type_columns

df, problemas_1 = read_and_clean_csv_mixed_types(FILE_PATH_1, nrows=100_000)
#loans_rejected, problemas_2 = read_and_clean_csv_mixed_types(FILE_PATH_2, nrows=100_000)

print(df.shape)
display(df.head(3))

# print(loans_rejected.shape)
# display(loans_rejected.head(3))

(100000, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
trail = AuditTrail(
    track_histograms=True,
    track_distributions=True,
    enable_logging=True,
    auto_detect_types=True,
    #target_col='target',
    default_keys=["id"]
)

trail.take_snapshot(df, name="original")

In [4]:
trail.describe_snapshot("original")


📄 Descrição do snapshot 'original':

▶️ Shape: (100000, 151)
▶️ Chaves de duplicação: ['id']
   • Duplicatas nas chaves: 0

🧱 Tipos de dados:


id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
funded_amnt_inv          float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object


🔎 Colunas detectadas automaticamente:
   • Numéricas (89): ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'char

member_id                                     100000
revol_bal_joint                               100000
sec_app_fico_range_high                       100000
sec_app_earliest_cr_line                      100000
sec_app_inq_last_6mths                        100000
sec_app_mort_acc                              100000
sec_app_open_acc                              100000
sec_app_revol_util                            100000
sec_app_open_act_il                           100000
sec_app_num_rev_accts                         100000
sec_app_chargeoff_within_12_mths              100000
sec_app_collections_12_mths_ex_med            100000
sec_app_mths_since_last_major_derog           100000
sec_app_fico_range_low                        100000
desc                                           99993
dti_joint                                      99500
annual_inc_joint                               99498
verification_status_joint                      99498
orig_projected_additional_accrued_interest    


📊 Estatísticas numéricas:


,count,mean,std,min,25%,50%,75%,max
id,100000.0,6.532613e+07,2.437069e+06,56705.00,63867535.25,65644568.00,66542848.00,68617057.00
member_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,100000.0,1.505586e+04,8.641837e+03,1000.00,8000.00,13875.00,20000.00,35000.00
funded_amnt,100000.0,1.505586e+04,8.641837e+03,1000.00,8000.00,13875.00,20000.00,35000.00
funded_amnt_inv,100000.0,1.504711e+04,8.636873e+03,900.00,8000.00,13825.00,20000.00,35000.00
...,...,...,...,...,...,...,...,...
hardship_payoff_balance_amount,781.0,1.024605e+04,6.362472e+03,55.73,5044.21,9195.99,14667.45,29401.04
hardship_last_payment_amount,781.0,1.874736e+02,1.794104e+02,0.02,46.50,138.31,273.84,926.41
settlement_amount,2949.0,4.935599e+03,3.494121e+03,186.00,2111.63,4251.72,6898.96,23105.58
settlement_percentage,2949.0,4.716637e+01,5.713345e+00,20.00,45.00,45.00,50.00,97.66



🏷️ Estatísticas categóricas:


,count,unique,top,freq
term,100000,2,36 months,68053
grade,100000,7,B,30772
sub_grade,100000,35,B3,6601
emp_title,93867,37529,Teacher,1948
emp_length,93888,11,10+ years,33591
home_ownership,100000,4,MORTGAGE,49477
verification_status,100000,3,Source Verified,42260
issue_d,100000,3,Dec-2015,44343
loan_status,100000,7,Fully Paid,70288
pymnt_plan,100000,2,n,99985



📈 Histogramas (categorias apenas):
  term: 2 valores distintos (top 3: {' 36 months': 68053, ' 60 months': 31947})
  grade: 7 valores distintos (top 3: {'B': 30772, 'C': 29198, 'A': 17493})
  sub_grade: 35 valores distintos (top 3: {'B3': 6601, 'B4': 6561, 'C1': 6457})
  emp_length: 12 valores distintos (top 3: {'10+ years': 33591, '< 1 year': 8765, '2 years': 8706})
  home_ownership: 4 valores distintos (top 3: {'MORTGAGE': 49477, 'RENT': 39257, 'OWN': 11265})
  verification_status: 3 valores distintos (top 3: {'Source Verified': 42260, 'Verified': 31798, 'Not Verified': 25942})
  issue_d: 3 valores distintos (top 3: {'Dec-2015': 44343, 'Nov-2015': 37360, 'Oct-2015': 18297})
  loan_status: 7 valores distintos (top 3: {'Fully Paid': 70288, 'Charged Off': 17603, 'Current': 11402})
  pymnt_plan: 2 valores distintos (top 3: {'n': 99985, 'y': 15})
  purpose: 12 valores distintos (top 3: {'debt_consolidation': 56914, 'credit_card': 25312, 'home_improvement': 6283})
  title: 14 valores dist

#### Limpeza com Vassoura

In [ ]:
vs = Vassoura(
    df,
    target_col="id",
    
    heuristics=[
        "graph_cut",
        "corr",
        "vif",
        "iv",
    ],
    
    thresholds={
        "missing":0.60,     # remove colunas com mais de 60% de missing
        "corr": 0.85,       # remove
        "vif": 5,
        "iv": 0.01,
        "graph_cut": 0.9,   # limiar para graph-cut
    },
)
df_limpo = vs.run(recompute=True)

[Vassoura] Missing heuristic (thr>0.6)
  → dropped ['member_id', 'desc', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_len

INFO | term -> categórica (2 categorias)
INFO | int_rate -> numérica
INFO | installment -> numérica
INFO | grade -> categórica (7 categorias)
INFO | sub_grade -> categórica (35 categorias)
INFO | emp_title ignorada (muitas categorias: 37529)
INFO | emp_length -> categórica (11 categorias)
INFO | home_ownership -> categórica (4 categorias)
INFO | annual_inc -> numérica
INFO | verification_status -> categórica (3 categorias)
INFO | issue_d -> categórica (3 categorias)
INFO | loan_status -> categórica (7 categorias)
INFO | pymnt_plan -> categórica (2 categorias)
INFO | url ignorada (muitas categorias: 100000)
INFO | purpose -> categórica (12 categorias)
INFO | title -> categórica (13 categorias)
INFO | zip_code ignorada (muitas categorias: 880)
INFO | addr_state -> categórica (49 categorias)
INFO | dti -> numérica
INFO | delinq_2yrs -> numérica
INFO | earliest_cr_line ignorada (muitas categorias: 620)
INFO | fico_range_high -> numérica
INFO | inq_last_6mths -> numérica
INFO | mths_since_l

  → dropped ['out_prncp', 'total_pymnt', 'recoveries', 'fico_range_low', 'num_actv_rev_tl', 'funded_amnt', 'tot_cur_bal', 'funded_amnt_inv', 'loan_amnt', 'open_acc', 'total_pymnt_inv'] (graph_cut>0.9)
[Vassoura] Corr heuristic (thr=0.85)


INFO | last_pymnt_amnt -> numérica
INFO | last_credit_pull_d -> categórica (42 categorias)
INFO | last_fico_range_high -> numérica
INFO | last_fico_range_low -> numérica
INFO | collections_12_mths_ex_med -> numérica
INFO | policy_code -> numérica
INFO | application_type -> categórica (2 categorias)
INFO | acc_now_delinq -> numérica
INFO | tot_coll_amt -> numérica
INFO | total_rev_hi_lim -> numérica
INFO | acc_open_past_24mths -> numérica
INFO | avg_cur_bal -> numérica
INFO | bc_open_to_buy -> numérica
INFO | bc_util -> numérica
INFO | chargeoff_within_12_mths -> numérica
INFO | delinq_amnt -> numérica
INFO | mo_sin_old_il_acct -> numérica
INFO | mo_sin_old_rev_tl_op -> numérica
INFO | mo_sin_rcnt_rev_tl_op -> numérica
INFO | mo_sin_rcnt_tl -> numérica
INFO | mort_acc -> numérica
INFO | mths_since_recent_bc -> numérica
INFO | mths_since_recent_inq -> numérica
INFO | num_accts_ever_120_pd -> numérica
INFO | num_actv_bc_tl -> numérica
INFO | num_bc_sats -> numérica
INFO | num_bc_tl -> num

  → dropped ['last_fico_range_low'] (corr>0.85)
  → dropped ['total_rev_hi_lim'] (corr>0.85)
  → dropped ['tot_hi_cred_lim'] (corr>0.85)
  → dropped ['bc_util'] (corr>0.85)
  → dropped ['total_bal_ex_mort'] (corr>0.85)
[Vassoura] VIF heuristic (thr=5)


INFO | int_rate -> numérica
INFO | installment -> numérica
INFO | grade -> categórica (7 categorias)
INFO | sub_grade -> categórica (35 categorias)
INFO | emp_title ignorada (muitas categorias: 37529)
INFO | emp_length -> categórica (11 categorias)
INFO | home_ownership -> categórica (4 categorias)
INFO | annual_inc -> numérica
INFO | verification_status -> categórica (3 categorias)
INFO | issue_d -> categórica (3 categorias)
INFO | loan_status -> categórica (7 categorias)
INFO | pymnt_plan -> categórica (2 categorias)
INFO | url ignorada (muitas categorias: 100000)
INFO | purpose -> categórica (12 categorias)
INFO | title -> categórica (13 categorias)
INFO | zip_code ignorada (muitas categorias: 880)
INFO | addr_state -> categórica (49 categorias)
INFO | dti -> numérica
INFO | delinq_2yrs -> numérica
INFO | earliest_cr_line ignorada (muitas categorias: 620)
INFO | fico_range_high -> numérica
INFO | inq_last_6mths -> numérica
INFO | mths_since_last_delinq -> numérica
INFO | pub_rec -> 

[Vassoura] Erro no cálculo de VIF — pulando heurística.
[Vassoura] IV heuristic (thr<0.01) – removendo low IV


C:\Users\JM\Documents\0_CienciaDados\1_Frameworks\Vassoura\vassoura\core.py:46: UserWarning: Target must be binary (0/1) for IV calculation.
  warnings.warn("Target must be binary (0/1) for IV calculation.")
C:\Users\JM\Documents\0_CienciaDados\1_Frameworks\Vassoura\vassoura\core.py:46: UserWarning: Target must be binary (0/1) for IV calculation.
  warnings.warn("Target must be binary (0/1) for IV calculation.")
C:\Users\JM\Documents\0_CienciaDados\1_Frameworks\Vassoura\vassoura\core.py:46: UserWarning: Target must be binary (0/1) for IV calculation.
  warnings.warn("Target must be binary (0/1) for IV calculation.")
C:\Users\JM\Documents\0_CienciaDados\1_Frameworks\Vassoura\vassoura\core.py:46: UserWarning: Target must be binary (0/1) for IV calculation.
  warnings.warn("Target must be binary (0/1) for IV calculation.")
C:\Users\JM\Documents\0_CienciaDados\1_Frameworks\Vassoura\vassoura\core.py:46: UserWarning: Target must be binary (0/1) for IV calculation.
  warnings.warn("Target mus

MemoryError: Unable to allocate 14.0 GiB for an array with shape (3752900000,) and data type int32

#### Comparação após limpeza

In [ ]:
trail.compare_snapshots("original", "filtrado")

In [ ]:


print(len(loans_accepted))
loans_accepted.dropna(axis=0, inplace=True)
print(len(loans_accepted))

In [ ]:
loans_accepted.term.value_counts(dropna=False, normalize=True)*100

In [ ]:
for col in df.columns:
    tipos = df[col].apply(type).value_counts()
    if len(tipos) > 1:
        print(f"{col}: {tipos}")

In [ ]:
vsess = Vassoura(
    df,
    target_col='ever90m12',
    heuristics=['corr', 'vif'],
    thresholds={'corr': 0.9, 'vif': 10},
)
df_clean = vsess.run()
df_clean.head()

In [ ]:
# Gera relatório resumido
vsess.generate_report('example_report.html')